# CNN - ResNet
This Program is using cats breed dataset with around 50 labels.
There are two source that being used in this program. The good one from robots, and the worse is from keagle dataset.
This prgram is using transferlearning with ResNet.
## Outline
- Import libraries
- Import dataset
- Split dataset
- Data Augmentation
- Model structuring
- Model Training
- Visualisasi Hasil

## Import Libraries


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [3]:
# import libraries
import os
import shutil
import numpy as np
import re
import matplotlib.pyplot as plt
import random

In [4]:
print(f"Using TensorFlow version: {tf.__version__}")
print(f"Using TensorFlow Dataset version: {tfds.__version__}")
print(f"Using TensorFlow Hub version: {hub.__version__}")
print(f"Using Numpy version: {np.__version__}")
print(f"GPU Device Found." if tf.config.list_physical_devices('GPU') else "GPU Device Not Found. Running on CPU")

Using TensorFlow version: 2.8.0
Using TensorFlow Dataset version: 4.5.2+nightly
Using TensorFlow Hub version: 0.12.0
Using Numpy version: 1.22.2
GPU Device Found.


## Import Datasets
importing dataset from local folder and also preparing stuff for data splitting

### Define path
there are 3 option of dataset to use in this program. From Kaggle, Robots, and the merged one

In [5]:
path_source = r"../Dataset/"
dataset_all = r"Merged Dataset All/"
dataset_kaggle = r"Merged Dataset Kaggle/"
dataset_robot = r"Robots Datasets/"

### Dataset review
Chekking how many dataset classes and how many image per classes. dataset infomation is stored in dictionary.

In [6]:
def dataset_review(dataset_type):
    """Checking what directory exist in datataset directory

    Args:
        dataset_type (string): root path for choosen dataset

    Returns:
        dictionary: store dictionary name as key and sum of image in it
    """
    dataset_path = os.path.join(path_source, dataset_type)
    folders = {}
    
    for folder in os.listdir(dataset_path):
        folders[folder] = len(os.listdir(os.path.join(dataset_path, folder)))
    
    return folders

In [7]:
datasetDict = dataset_review(dataset_all)

temp = 0
for key in datasetDict:
    temp += datasetDict[key]

print(f"There are {len(datasetDict)} classes with {temp} images in total", end="\n\n")
for key, value in datasetDict.items():
    print(f"class {key} have {value} images")

There are 58 classes with 124386 images in total

class Abyssinian have 428 images
class American Bobtail have 919 images
class American Curl have 139 images
class American Shorthair have 4822 images
class Applehead Siamese have 130 images
class Balinese have 248 images
class Bengal have 2544 images
class Birman have 420 images
class Bombay have 1919 images
class British Shorthair have 753 images
class Burmese have 335 images
class Calico have 3345 images
class catpedigree_merged have 6 images
class Chartreux have 81 images
class Cornish Rex have 159 images
class Devon Rex have 112 images
class Dilute Calico have 3121 images
class Dilute Tortoiseshell have 3015 images
class Domestic Long Hair have 4382 images
class Domestic Medium Hair have 5323 images
class Domestic Short Hair have 52105 images
class Egyptian Mau have 500 images
class Exotic Shorthair have 465 images
class Extra-Toes Cat - Hemingway Polydactyl have 1154 images
class Havana have 166 images
class Himalayan have 1264 ima

### Directory Preparation
Creating training, testing, and/or validation directory for storing dataset split later.

In [8]:
def create_dir(rooth_path, folders):
    if os.path.isdir(os.path.join(rooth_path, "training")):
        print(f"this path {os.path.join(rooth_path, 'training')} already made")
    else:
        os.makedirs(os.path.join(rooth_path, "training"))
        for folder in folders.keys():
            os.makedirs(os.path.join(rooth_path, f"training/{folder}"))
    
    if os.path.isdir(os.path.join(rooth_path, "testing")):
        print(f"this path {os.path.join(rooth_path, 'testing')} already made")
    else:
        os.makedirs(os.path.join(rooth_path, "testing"))
        for folder in folders.keys():
            os.makedirs(os.path.join(rooth_path, f"testing/{folder}"))
    

In [9]:
create_dir(os.path.join(path_source, "Used"), datasetDict)

this path ../Dataset/Used\training already made
this path ../Dataset/Used\testing already made


In [10]:
def split_dataset(sourcePath, trainPath, testPath, splitSize):
    files = []
    
    for file_name in os.listdir(sourcePath):
        if os.path.getsize(os.path.join(sourcePath, file_name)):
            files.append(file_name)
        else:
            print(f"{file_name} is ignored")
    
    split_point = int(splitSize * len(files))
    shuffled_file = random.sample(files, len(files))
    
    trainFile = shuffled_file[:split_point]
    testFile = shuffled_file[split_point:]
    
    for file_train in trainFile:
        shutil.copyfile(os.path.join(sourcePath, file_train),
                        os.path.join(trainPath, re.sub(r"([\s/-]+)", r"_", file_train).lower()))
    for file_test in testFile:
        shutil.copyfile(os.path.join(sourcePath, file_test),
                        os.path.join(testPath, re.sub(r"([\s/-]+)", r"_", file_test).lower()))

In [11]:
for folder in os.listdir(os.path.join(path_source, dataset_all)):
    train_dir = os.path.join(f"{path_source}/Used", f"training/{folder}")
    test_dir = os.path.join(f"{path_source}/Used", f"testing/{folder}")
    
    try:
        split_dataset(os.path.join(f"{path_source}/{dataset_all}", folder), train_dir, test_dir, 0.9)
        print(f"folder {folder} image copy succes.")
    except:
        print(f"folder {folder} copy is failed", "===========")

folder Abyssinian image copy succes.
folder American Bobtail image copy succes.
folder American Curl image copy succes.
folder American Shorthair image copy succes.
folder Applehead Siamese image copy succes.
folder Balinese image copy succes.
folder Bengal image copy succes.
folder Birman image copy succes.
folder Bombay image copy succes.
folder British Shorthair image copy succes.
folder Burmese image copy succes.
folder Calico image copy succes.
dummy_data is ignored
__pycache__ is ignored
folder catpedigree_merged image copy succes.
folder Chartreux image copy succes.
folder Cornish Rex image copy succes.
folder Devon Rex image copy succes.
folder Dilute Calico image copy succes.
folder Dilute Tortoiseshell image copy succes.
folder Domestic Long Hair image copy succes.
folder Domestic Medium Hair image copy succes.
folder Domestic Short Hair copy is failed ===========
folder Egyptian Mau copy is failed ===========
folder Exotic Shorthair copy is failed ===========
